In [1]:
import cobra
from cobra.io.mat import *
import warnings
warnings.simplefilter('ignore')
#from cobra.sampling import sample
from cobra.sampling import OptGPSampler, ACHRSampler
import os
cpus = os.cpu_count()
from cobra.io.json import load_json_model
model = load_matlab_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.mat")
print(
model.optimize(),
cpus)

This model seems to have metCharge instead of metCharges field. Will use metCharge for what metCharges represents.
This model seems to have metHMDB instead of metHMDBID field. Will use metHMDB for what metHMDBID represents.
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, l, m, n, r, x


<Solution 3.198 at 0x7f477fad7c70> 20


In [2]:
import re
from itertools import compress



rxns        = [ r.id for r in model.reactions]
subsystems  = [s.subsystem for s in model.reactions]
subsys_bool = [bool(re.search("Phen|phen|Tetrahydrobiopterin", s)) for s in subsystems]
subsys_rxns = list(compress(rxns, subsys_bool))

reversible = [
'PHETA1',
'PHETA1m',
'r0547',
'RE0830C',
'RE0830N',
'RE1709C',
'RE1709N',
'RE2660C',
'RE2660N']

forward_rxns =  list(set(subsys_rxns)-set(reversible))

for r in forward_rxns:
    model.reactions.get_by_id(r).bounds  = (1e-8, 10)
    
    
for r in reversible:
    model.reactions.get_by_id(r).bounds  = (-10, 10)

co    = model.problem.Constraint(model.reactions.biomass.flux_expression,
                                 lb=.01 * model.optimize().objective_value,
                                 ub=.99 * model.optimize().objective_value)
model.add_cons_vars([co])
assert model.optimize().objective_value > 0

In [7]:
optgp = OptGPSampler(model, processes=cpus, thinning=1000)
samples = optgp.sample(10_000)
samples.to_parquet("./results/flux/flux_samples.parquet.gzip", compression='gzip')  

True

### Get non zero samples from phenylalanine and biopterin subsystems

In [1]:
import pandas as pd
from cobra.io.mat import *
from itertools import compress

model = load_matlab_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.mat")
samples = pd.read_parquet("./results/flux/flux_samples.parquet.gzip")  
rxns        = [ r.id for r in model.reactions]
subsystems  = [s.subsystem for s in model.reactions]
subsys_bool = [bool(re.search("Phen|phen|Tetrahydrobiopterin", s)) for s in subsystems]
subsys_rxns = list(compress(rxns, subsys_bool))

This model seems to have metCharge instead of metCharges field. Will use metCharge for what metCharges represents.
This model seems to have metHMDB instead of metHMDBID field. Will use metHMDB for what metHMDBID represents.
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, l, m, n, r, x


In [24]:

samples_subsys_rxns = samples[subsys_rxns]
samples_subsys_rxns.loc[:,
list(samples_subsys_rxns.sum().abs() > 1e-7 )].to_parquet("./results/flux/subsys_fluxes_non_zero.parquet.gzip")
